In [4]:
#!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
#!pip install ftfy regex tqdm
#!pip install git+https://github.com/openai/CLIP.git

import torch
import pandas as pd
#from google.colab import drive
import tarfile
import os
from PIL import Image
import clip
import numpy as np

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
drive.mount('/content/drive')

# drive/My Drive/Deep Learning Project/refcocog.tar.gz

if not os.path.isdir('drive/My Drive/Deep Learning Project/refcocog'):
    print("Extracting dataset")
    tar = tarfile.open('drive/My Drive/Deep Learning Project/refcocog.tar.gz', 'r:gz')
    tar.extractall(path='drive/My Drive/Deep Learning Project/refcocog')
    tar.close()
else:
    print("Dataset already extracted")

In [5]:
print(clip.available_models())
# ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'ViT-B/32', 'ViT-B/16']

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("RN50", device=device)
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

# Load dataset




# image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

# with torch.no_grad():
#     image_features = model.encode_image(image)
#     text_features = model.encode_text(text)
    
#     logits_per_image, logits_per_text = model(image, text)
#     probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

100%|███████████████████████████████████████| 244M/244M [01:10<00:00, 3.62MiB/s]


AssertionError: Torch not compiled with CUDA enabled

This dataset consists of 85,474 referring expressions for 54,822
objects in 26,711 images. Images were selected to contain between 2 and 4 objects
of the same object category.

In [ ]:
import pickle
from torch.utils.data import Dataset

class CocoGDataset(Dataset):
    
    def __init__(self, annotations, imgs, size):
        super(CocoGDataset, self).__init__()
        self.annotations = annotations
        
        self.imgs = imgs
        self.size = size

    def __getitem__(self, idx):
        return torch.tensor([idx], dtype=torch.float32)
    
    def __len__(self):
        return self.size
    
size = 4
dataset = CocoGDataset(size)

length = len(dataset)
print(f"Dataset length: {length}")

# get each element of the dataset through indexing
# (__getitem__ method)
for idx in range(len(dataset)):
    print(f"- {idx}: {dataset[idx]}")

In [ ]:
import pickle

file_path = 'drive/My Drive/Deep Learning Project/refcocog/refcocog/annotations/refs(umd).p'

file = open(file_path, 'rb')

# Load pickle data
data = pickle.load(file)

# close the file
file.close()

```javascript
{'image_id': 545022,
  'split': 'test',
  'sentences': [{'tokens': ['the',
     'horse',
     'that',
     'can',
     'barely',
     'be',
     'seen'],
    'raw': 'The horse that can barely be seen.',
    'sent_id': 12541,
    'sent': 'the horse that can barely be seen'},
   {'tokens': ['the',
     'back',
     'leftmost',
     'horse',
     'who',
     "'",
     's',
     'ears',
     'you',
     'can',
     'see',
     'above',
     'the',
     'other',
     'horse',
     "'",
     's',
     'butt'],
    'raw': "The back leftmost horse who's ears you can see above the other horse's butt",
    'sent_id': 12542,
    'sent': "the back leftmost horse who ' s ears you can see above the other horse ' s butt"}],
  'file_name': 'COCO_train2014_000000545022_60678.jpg',
  'category_id': 19,
  'ann_id': 60678,
  'sent_ids': [12541, 12542],
  'ref_id': 551}
```

instances.json

```javascript
{'segmentation': [[591.38,
     225.26,
     609.04,
     232.67,
     640.0,
     238.4,
     639.73,
     221.22,
     640.0,
     220.81,
     640.0,
     218.63,
     627.87,
     219.17,
     595.5,
     218.69]],
   'area': 676.7527500000007,
   'iscrowd': 0,
   'image_id': 735,
   'bbox': [591.38, 218.63, 48.62, 19.77],
   'category_id': 51,
   'id': 1898502},
```